In [1]:
import numpy as np
import pickle
import json
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize, RegexpTokenizer
from pyfasttext import FastText
from collections import defaultdict, Counter
import string
from statistics import mean

In [2]:
model = FastText("/data/rali7/Tmp/solimanz/data/wikipedia/wiki.en.bin")

In [3]:
with open("/data/rali7/Tmp/solimanz/data/datasets/top550/jobid/data.json", "rb") as f:
    data550 = json.load(f)
with open("/data/rali7/Tmp/solimanz/data/datasets/reduced7000/jobid/data.json", "rb") as f:
    data7k = json.load(f)

In [13]:
with open("/part/01/Tmp/solimanz/data/skill_lists.json", "r") as f:
    skills = f.read()

In [17]:
skills = skills.replace("}", "},")
skills = skills[:-2] + '\n'
skills = "[" + skills + "]"
skills = json.loads(skills)

In [4]:
tokenizer = RegexpTokenizer(r'\s+|\W', gaps=True)

In [40]:
id_skills = {doc['id']: [skill.lower() for skill in doc['skills']] for doc in skills}

In [43]:
pickle.dump(id_skills, open('/data/rali7/Tmp/solimanz/data/pickles/skills.pkl', 'wb'))

In [10]:
id_skills = pickle.load(open('/data/rali7/Tmp/solimanz/data/pickles/skills.pkl', 'rb'))

In [8]:
ids550 = [p[0] for p in data550['train_data'] + data550['test_data']]
ids7k = [p[0] for p in data7k['train_data'] + data7k['test_data']]

In [22]:
id_skills550 = dict()
for id_ in ids550:
    id_skills550[id_] = list(set(id_skills[id_]))

In [23]:
def get_token_ids(id_skills):
    sw = set(stopwords.words('english') + ['...'])
    tokenizer = RegexpTokenizer(r'\s+|\W', gaps=True) 
    skills = []    
    for s in id_skills.values():
        skills += s
    
    tokens = list(set(tokenizer.tokenize(" ".join(skills))))
    tokens = [t for t in tokens if t not in string.punctuation and t not in sw]
    
    return {token: id_ for id_, token in enumerate(tokens)}

In [24]:
def get_skill_tokens(id_skills):    
    tokenizer = RegexpTokenizer(r'\s+|\W', gaps=True)    
    for key, skills in id_skills.items():
        tokens = tokenizer.tokenize(" ".join(skills))
        id_skills[key] = tuple(set(tokens))
        
    return id_skills

In [25]:
token_id = get_token_ids(id_skills550)

In [40]:
token_id['<NULL>'] = len(token_id)

In [44]:
token_id['<NULL>']

20209

In [26]:
id_skills550 = get_skill_tokens(id_skills550)

In [27]:
skills_embs = np.zeros((len(token_id), 300), dtype=np.float32)

In [29]:
for skill in token_id.keys():    
    emb = model.get_numpy_vector(skill)
    skills_embs[token_id[skill]] = emb

In [54]:
skills_embs = np.vstack((skills_embs, np.zeros((300), dtype=np.float32)))

In [76]:
skills_embs.shape

(20210, 300)

In [56]:
np.save("/data/rali7/Tmp/solimanz/data/datasets/top550/skill_embs/skill_tokens.npy", skills_embs)

In [35]:
id_sid = {id_: [token_id[tok] for tok in skills if tok in token_id] for id_, skills in id_skills550.items()}

In [58]:
for k, v in id_sid.items():
    if not v:
        id_sid[k].append('<NULL>')

In [84]:
token_id['<NULL>']

20209

In [59]:
train_data = data550['train_data']
test_data = data550['test_data']

In [64]:
for dat in train_data:
    id_ = dat[0]
    dat.append(id_sid[id_])

In [66]:
for dat in test_data:
    id_ = dat[0]
    dat.append(id_sid[id_])

In [103]:
data550['train_data'] = train_data
data550['test_data'] = test_data
data550['skill_token_id'] = token_id

In [70]:
data = train_data + test_data
data550['max_skills_num'] = max([len(d[-1]) for d in data])

In [79]:
data550['skills_embs'] = "/data/rali7/Tmp/solimanz/data/datasets/top550/skill_embs/skill_tokens.npy"

In [104]:
with open('/data/rali7/Tmp/solimanz/data/datasets/top550/skill_embs/data.json', 'w') as f:
    json.dump(data550, f)

In [88]:
for dat in test_data:
    if dat[2][0] == '<NULL>':
        dat[2][0] = token_id['<NULL>']

In [93]:
data550.keys()

dict_keys(['title_to_id', 'train_data', 'test_data', 'maximum_seq_len', 'skill_token_id', 'max_skills_num', 'skills_embs'])

In [94]:
data550['max_skills_num']

207

In [100]:
for dat in train_data:
    if len(dat[2]) < data550['max_skills_num']:
        for i in range(data550['max_skills_num'] - len(dat[2])):
            dat[2].append(token_id['<NULL>'])

In [102]:
np.unique([len(dat[2]) for dat in train_data])

array([207])